<a href="https://colab.research.google.com/github/WilderGitHub/datascience/blob/main/v3scrappingLARAZON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instalación e importación de librerias necesarias

In [ ]:
!pip install beautifulsoup4


In [ ]:
from ipaddress import collapse_addresses
from os import sep
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
from requests import get
import locale
from datetime import datetime as dt

In [ ]:
desde = 4
hasta = 5
pasito = 1
columnas = ["codigo", "fecha", "titular", "detalle", "texto"]
tupla = ("enero","febrero","marzo","abril","mayo","junio","julio",
        "agosto","septiembre","octubre","noviembre","diciembre")

consolidado = pd.DataFrame()
#pagina=3500

In [ ]:
consolidado

""


### Con requests obtenemos el contenido de la url y procesamos


In [ ]:
import requests
url = 'https://www.la-razon.com/economia/2023/02/21/una-falla-en-un-avion-de-ecojet-paraliza-los-vuelos-en-el-alto-por-casi-cuatro-horas/'
session = requests.Session()
response = session.get(url)
print(response)

<Response [403]>


In [ ]:
for pagina in range(desde, hasta+1, pasito):
  # Obtenemos los links de una pagina en una lista (listalinks)

  url = "https://www.la-razon.com/economia/page/" + str(pagina)+"/"
  print("uriel:",url)
  r = requests.get(url)
  print("la erre:",r)
  r = r.content.decode('utf-8', 'ignore')
  soup = BeautifulSoup(r, 'html.parser')

  # Encontramos el cuerpo principal de los articulos
  arts = soup.find('div', id='lr-main')
  ########################################################
  # Encontramos los links de los articulos
  links = arts.find_all('a', class_="link", limit=5)
  listalinks = []
  for a in links:
    if 'href' in a.attrs:
      listalinks.append(a['href'])
  ##print(listalinks)
  ids = []
  fechas = []
  titulares = []
  detalles = []
  textos = []

  for n in range(len(links)):
    r = requests.get(listalinks[n])
    r = r.content.decode('utf-8', errors='ignore')
    articulos = BeautifulSoup(r, 'lxml')  # html5lib')
    elarticulo = articulos.find('div', id="lr-main")
    # print("el articulo: ", type(elarticulo))
    try:
        indice = elarticulo.find_next(id).find_next(id).find_next(id).find_next(id)
        ids.append(indice['id'])
        #print("tryando",indice['id'])

        fecha = elarticulo.find_all('span', limit=2)[1].text
        deit = re.search('(\d{2}) de (\w+) de (\d{4})', fecha)
        mesesito = tupla.index(deit.group(2))+1
        mifechaaux=deit.group(1)+str(mesesito)+deit.group(3)
        mifecha = dt.strptime(mifechaaux, '%d%m%Y')

        titular = elarticulo.find_next("h1").text
        detalle = elarticulo.find_next('p').text

        divsparrafos = elarticulo.findNext('div', class_="article-body")
        parrafos = divsparrafos.findChildren('p')


    except:
        indice = elarticulo.find_next(id)
        # print("a ver",indice)
        ids.append("n.d.")

        try:
          fecha = elarticulo.find_all('span', limit=1)[0].text
          deit = re.search('(\d{2}) de (\w+) de (\d{4})', fecha)
          mesesito = tupla.index(deit.group(2))+1
          mifechaaux=deit.group(1)+str(mesesito)+deit.group(3)
          mifecha = dt.strptime(mifechaaux, '%d%m%Y')
        except:
          try:
            fecha = elarticulo.find_all('span', limit=2)[0].text
            deit = re.search('(\d{2}) de (\w+) de (\d{4})', fecha)
            mesesito = tupla.index(deit.group(2))+1
            mifechaaux=deit.group(1)+str(mesesito)+deit.group(3)
            mifecha = dt.strptime(mifechaaux, '%d%m%Y')
          except:
            #fecha = elarticulo.find_all('span', limit=1)[0].text
            mifecha = "/ 17 de enero de 1977"


        titular = elarticulo.find_next("p", class_="special-art-title")
        detalle = elarticulo.find_next('p').text

        divsparrafos = elarticulo.findNext('div', class_="article-body")
        parrafos = divsparrafos.findChildren('p')
        parrafos = divsparrafos.findChildren('p')
    else:
      indice = elarticulo.find_next(id)
      # print("a ver",indice)
      ids.append("n.d.")

      fecha = elarticulo.find_all('span', limit=2)[1].text
      deit = re.search('(\d{2}) de (\w+) de (\d{4})', fecha)
      mesesito = tupla.index(deit.group(2))+1
      mifechaaux=deit.group(1)+str(mesesito)+deit.group(3)
      mifecha = dt.strptime(mifechaaux, '%d%m%Y')


    titulares.append(titular)
    detalles.append(detalle)
    fechas.append(mifecha)

    parrafoses = []
    for pp in parrafos:
        parrafoses.append(pp.string)
        # .replace("</p> <p>"," ").replace("<p>","").replace("</p>","")
        unparrafo = ' '.join(map(str, parrafoses))
    textos.append(unparrafo)

  troso = pd.DataFrame(list(zip(ids, fechas, titulares, detalles, textos)), columns=columnas)

  consolidado = consolidado.append(troso, ignore_index=True)

uriel: https://www.la-razon.com/economia/page/4/
la erre: <Response [403]>


AttributeError: ignored

In [ ]:
consolidado.head(100)

,codigo,fecha,titular,detalle,texto
0,article145992,2012-09-18 00:00:00,Presidente Morales pide «convivir» a cooperati...,Morales dijo que antes la FSTMB enfrentaba dic...,"El presidente del Estado, Evo Morales demandó ..."
1,n.d.,2012-09-18 00:00:00,Muere un minero asalariado y el Gobierno convo...,El Gobierno informó que nueve personas resulta...,El minero asalariado Héctor Choque falleció es...
2,article146006,2012-09-18 00:00:00,Envían a la cárcel a mineros cooperativistas q...,El juez explicó que la portación y la utilizac...,El juez Séptimo de Instrucción en lo Penal de ...
3,n.d.,2012-09-18 00:00:00,"El BID premia innovación agrícola en Bolivia, ...",Se trata de la tercera edición del programa MA...,El Banco Interamericano de Desarrollo (BID) y ...
4,article145997,2012-09-18 00:00:00,Cooperativistas piden militariza Colquiri y am...,Dicen que fueron provocados y anuncian que res...,Tras la violenta arremetida de los mineros coo...
...,...,...,...,...,...
95,article144800,/ 17 de enero de 1977,None,Santa Cruz y La Paz son las regiones que lider...,El director general del Instituto Nacional de ...
96,n.d.,/ 17 de enero de 1977,None,Ambos productos se destinan al consumo interno...,"En el territorio nacional, los principales pro..."
97,article144801,/ 17 de enero de 1977,None,Advertencia. Los trabajadores asalariados amen...,Los mineros sindicalizados y los cooperativist...
98,n.d.,/ 17 de enero de 1977,None,Contrato. La demanda se debe a la nueva adenda,"En agosto, la comercialización de gas natural ..."


In [ ]:
consolidado.to_csv("prueba"+str(desde)+"-"+str(hasta) +
                   ".csv", encoding='utf8', sep="|")
#print("Yasta terminado oe")

Yasta terminado oe
